In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/My Drive/ML/train_resized_299_299.zip'

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pathlib
import PIL
from PIL import Image
import os

In [ ]:
print(os.listdir())

['.config', 'train_resized_299_299', 'drive', 'sample_data']


In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory = '/content/train_resized_299_299',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(299, 299),
    shuffle=True,
    seed=300,
    validation_split=0.2,
    subset='training',
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False)

Found 15653 files belonging to 40 classes.
Using 12523 files for training.


In [ ]:
validation_ds = tf.keras.utils.image_dataset_from_directory(
    directory = '/content/train_resized_299_299',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(299, 299),
    shuffle=True,
    seed=300,
    validation_split=0.2,
    subset='validation',
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False)

Found 15653 files belonging to 40 classes.
Using 3130 files for validation.


In [ ]:
conv_base = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(299, 299, 3),
    pooling=None,
    classes=1000,
    classifier_activation="softmax")

In [ ]:
conv_base.trainable = False

In [ ]:
data_augmentation = tf.keras.Sequential([tf.keras.layers.RandomFlip("horizontal_and_vertical"), tf.keras.layers.RandomRotation(factor=0.2),
                                         tf.keras.layers.RandomZoom(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2))])

In [ ]:
inputs = tf.keras.Input(shape=(299, 299, 3))
x = data_augmentation(inputs)
x = tf.keras.applications.xception.preprocess_input(x)
x = conv_base(x, training=False)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512)(x)
x = tf.keras.layers.Dense(256)(x)
x = tf.keras.layers.Dense(128)(x)
#x = layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(40, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer="adadelta", metrics=["accuracy"])
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath="/content/xception_feature_extraction_with_data_augmentation.keras", save_best_only=True, monitor="val_loss")]

In [ ]:
history = model.fit(train_ds, epochs=5, validation_data=validation_ds, callbacks=callbacks)
!cp "/content/xception_feature_extraction_with_data_augmentation.keras" "/content/drive/My Drive/ML/xception_feature_extraction_with_data_augmentation_299_299.keras"

In [ ]:
model = tf.keras.models.load_model("/content/drive/My Drive/ML/xception_feature_extraction_with_data_augmentation_299_299.keras")

In [ ]:
conv_base.trainable = True

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adadelta', metrics=["accuracy"])
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath="/content/fine_tuned_xception_feature_extraction_with_data_augmentation.keras", save_best_only=True, monitor="val_loss")]

In [ ]:
history = model.fit(train_ds, epochs=10, validation_data=validation_ds, callbacks=callbacks)
!cp "/content/fine_tuned_xception_feature_extraction_with_data_augmentation.keras" "/content/drive/My Drive/ML/fine_tuned_xception_feature_extraction_with_data_augmentation.keras_299_299.keras"